# Datenvorbereitung
Dieses Notebook dient der Extraktion und Aufbereitung von Daten aus dem TIGER-Korpus zur weiteren Verarbeitung. Die zugrunde liegende Datenvorbereitung orientiert sich an diesem https://github.com/theodm/gender-assistenz/blob/master/results/tiger_extract.py, wurde jedoch in mehreren Punkten überarbeitet und verbessert. Die Artikel werden automatisch segmentiert und einer eindeutigen Artikel-ID zugewiesen. Zudem erfolgt eine automatische Filterung der Autorinnen und Autoren, sodass dieser Schritt nicht mehr manuell durchgeführt werden muss.
## Datenauslesen
Im ersten Schritt werden die Daten aus der XML-Datei extrahiert. Dabei sollen Wörter, die als grammatisch männlich erkannt werden, durch zwei Bindestriche vor und nach dem Wort markiert werden. Die Texte werden artikelweise in eine Ausgabedatei geschrieben, um anschließend manuell weiterbearbeitet werden zu können. 

In [3]:
from lxml import etree as ET
import csv
import re
import pandas as pd
import spacy
import textdescriptives as td


Zunächst wird die Datei eingelesen, die eine Zuordnung der einzelnen Sätze zu ihren jeweiligen Artikeln enthält. Die Informationen werden in einem Dictionary gespeichert, um eine strukturierte Weiterverarbeitung zu ermöglichen.

In [4]:
sentences_to_documents_dict = {}
sentences_to_type_dict = {}
with open("sentences.tsv", "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter="\t")  
    for row in reader:
        if len(row) >= 3:  # Sicherstellen, dass die Zeile mindestens drei Spalten hat
            sentence_id = row[1]  # ID als Schlüssel (Spalte B)
            article_id = row[0]  # Erste Spalte als Wert (Spalte A)
            _type = row[2]
            sentences_to_documents_dict[sentence_id] = article_id
            sentences_to_type_dict[sentence_id] = _type
            

Anschließend wird die TIGER-XML-Datei eingelesen. Damit dieser Schritt erfolgreich ausgeführt werden kann, muss die Datei unter dem erwarteten Pfad verfügbar sein. Die enthaltenen Sätze werden dabei den zuvor zugeordneten Artikeln entsprechend sortiert abgespeichert. Potenziell generische Maskulina werden im Text durch die Markierung mit zwei Bindestrichen vor und nach dem betreffenden Wort kenntlich gemacht (z. B. --Lehrer--).

In [5]:
# Das ist die ID des ersten Artikels diese kann hier ausgelesen werden
previous_article = sentences_to_documents_dict["1"]
counter = 0
with open("preparedData.txt", "w", encoding="utf-8") as outputFile:
    for event, s_tag in ET.iterparse("tiger_release_aug07.corrected.16012013.xml", events=("end",), tag=f"s"):
        tag_id=s_tag.get("id")[1:]
        current_article = sentences_to_documents_dict[tag_id]
        if current_article != previous_article:
            outputFile.write(f"\n---\n\nArtikelId: {current_article}\n") 
            previous_article = current_article
        # Hinter diesem type versteckt sich der 
        if sentences_to_type_dict[tag_id] == "Meta":
            continue
        graph_tag = s_tag.find(f"graph")
        terminals_tag = graph_tag.find(f"terminals")
        t_tags = terminals_tag.findall(f"t")

        sentence = []
        for t_tag in t_tags:
            word = {}
    
            word["word"] = t_tag.attrib["word"]
            word["pos"] = t_tag.attrib["pos"]
            word["number"] = t_tag.attrib["number"]
            word["gender"] = t_tag.attrib["gender"]
            word["case"] = t_tag.attrib["case"]
    
            if (word["gender"] in ["Masc", "*"] or (word["gender"] in ["Neut"] and word["number"] == "Plur")) and word["pos"] in ["PDS", "PIS", "PPER", "PPOSS", "PRELS", "PWS", "NN"]:
                word["word"] = "--" + word["word"] + "--"
    
            sentence.append(word)
    
        outputFile.write(" ".join(x["word"] for x in sentence).replace(" ,", ",").replace(" ;", ";").replace(" .", ".").replace(" :", ":").replace(" ?", "?").replace(" .", ".").replace(" !", "!").replace("`` ", "\"").replace(" `` ", "\"").replace(" ``", "\"").replace("'' ", "\"").replace(" '' ", "\"").replace(" ''", "\"").replace("( ", "(").replace(" )", ")") + "\n")


FileNotFoundError: [Errno 2] No such file or directory: 'tiger_release_aug07.corrected.16012013.xml'

n der zuvor erstellten Datei erfolgt im nächsten Schritt eine manuelle Annotation. Dabei wird für jede markierte Wortform entschieden, ob es sich um ein generisches Maskulinum handelt und somit eine geschlechtergerechte Korrektur erforderlich ist. Als generisches Maskulinum gilt ein Ausdruck, wenn eine maskuline Form verwendet wird, um eine gemischtgeschlechtliche oder geschlechtsunspezifische Gruppe zu bezeichnen, ohne dass eine explizite geschlechtliche Markierung vorliegt. Typische Beispiele sind maskuline Substantive wie Student, Arzt oder Bürger, die in einem allgemeinen Sinn für alle Geschlechter stehen. Ausschlaggebend ist dabei der Gebrauchskontext: Liegt keine eindeutig geschlechtsspezifische Referenz vor und fungiert die maskuline Form als Standard, ist eine Korrektur in der Regel angezeigt. Die annotierten Formen werden entsprechend gekennzeichnet: Korrekturbedürftige Begriffe erhalten ein vorangestelltes Ausrufezeichen („!"), nicht korrekturbedürftige Formen ein umgekehrtes Schrägzeichen („\“).
## Weiterverarbeitung der manuellen Daten
Im folgenden Abschnitt werden die manuell annotierten Daten in ein DataFrame überführt. Dabei wird der ursprüngliche Fließtext ohne Markierungen rekonstruiert, um eine neutrale Vergleichsbasis zu gewährleisten. Zusätzlich erfolgt eine Segmentierung der Texte auf Satzebene: Die einzelnen Sätze jedes Artikels werden extrahiert und in Form einer separaten Liste gespeichert. Diese Struktur ermöglicht eine feingranulare Analyse sowie eine spätere Zuordnung von Annotationen und Systemergebnissen auf Satzebene.

In [6]:
with open('preparedData_manual.txt', 'r', encoding='utf-8') as f:
    content = f.read()

# Aufteilen in Blöcke anhand des Trennzeichen
artikel_blocks = content.strip().split('\n\n---\n\n')

data = []

for block in artikel_blocks:
    lines = block.strip().split('\n')
    artikel_id = lines[0]
    text = '\n'.join(lines[1:])  # Falls der Artikeltext mehrzeilig ist
    data.append((artikel_id, text))

df = pd.DataFrame(data, columns=['ArtikelId', 'Text'])


Der unmarkierte Volltext und satzweise Listen (markiert/unmarkiert) werden aus den manuell annotierten Daten generiert. Eine Prüfung stellt sicher, dass keine unmarkierten generisch-maskulinen Formen im Text verbleiben, bei Verstößen wird ein Fehler ausgelöst, um manuelle Nachkorrektur zu erzwingen.

In [7]:
def clean_and_check_article(article):
    #
    # Parst einen Artikel im
    #
    t = article
    ci = 0
    clean_article = ""


    number_of_ignored_chars = 0
    try:
        while ci < len(article)-2:
            if article[ci] in ("!", "\\") and article[ci+1:ci+3] == "--":
                # Zum Anfang des Wortes springen
                ci = ci + 3
                while True:
                    if article[ci:ci+2] == "--":
                        ci = ci + 2
                        break
                    clean_article = clean_article + article[ci]
                    ci = ci + 1
                continue
    
            if t[ci] == "-" and t[ci + 1] == "-":
                raise Exception("-- found: " + article[ci-10:ci+20])
    
            clean_article = clean_article + article[ci]
            ci = ci + 1
    
        return clean_article
    except: 
        print(article)
        raise Exception("-- found: " + article[ci-10:ci+20])



In [8]:
df['Text_unmarked'] = df['Text'].apply(clean_and_check_article)
df['Sentences_marked'] = df['Text'].apply(lambda x: x.split('\n'))
df['Sentences_unmarked'] = df['Text_unmarked'].apply(lambda x: x.split('\n'))

Um später detaillierte Analysen zu ermöglichen werden an dieser Stelle noch die dependency distance und die prop_adjacent distance berechnet.

In [10]:
nlp = spacy.load("de_core_news_lg")
nlp.add_pipe("textdescriptives/dependency_distance")  # fügt alle Features hinzu

def extract_features(text):
    doc = nlp(text)
    return {
        "dependency_distance": doc._.dependency_distance,
    }

features = df["Text_unmarked"].apply(extract_features)

df["dependency_distance"] = features.apply(lambda x: x["dependency_distance"])

# Optional: df.head() zur Kontrolle
df.head(5)

/opt/anaconda3/envs/spacy310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/spacy310/lib/python3.10/site-packages/IPython/core/formatters.py:406: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/opt/anaconda3/envs/spacy310/lib/python3.10/site-packages/IPython/core/formatters.py:406: FutureWarning: RangeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,ArtikelId,Text,Text_unmarked,Sentences_marked,Sentences_unmarked,dependency_distance
0,ArtikelId: 0001_0001,"""Ross Perot wäre vielleicht ein prächtiger \--...","""Ross Perot wäre vielleicht ein prächtiger Dik...","[""Ross Perot wäre vielleicht ein prächtiger \-...","[""Ross Perot wäre vielleicht ein prächtiger Di...",{'dependency_distance_mean': 2.965713889246949...
1,ArtikelId: 0001_0002,IBM und Siemens gelten nicht mehr als Schimpfw...,IBM und Siemens gelten nicht mehr als Schimpfw...,[IBM und Siemens gelten nicht mehr als Schimpf...,[IBM und Siemens gelten nicht mehr als Schimpf...,{'dependency_distance_mean': 2.782908048111762...
2,ArtikelId: 0001_0003,Wechselspiel von Dramatisierung und Ignoranz\n...,Wechselspiel von Dramatisierung und Ignoranz\n...,"[Wechselspiel von Dramatisierung und Ignoranz,...","[Wechselspiel von Dramatisierung und Ignoranz,...",{'dependency_distance_mean': 3.174690061989753...
3,ArtikelId: 0001_0004,Im \--Blickpunkt--:\nErmittlungen gegen \--Aut...,Im Blickpunkt:\nErmittlungen gegen Autonome\nS...,"[Im \--Blickpunkt--:, Ermittlungen gegen \--Au...","[Im Blickpunkt:, Ermittlungen gegen Autonome, ...",{'dependency_distance_mean': 3.137275306681283...
4,ArtikelId: 0001_0005,Für ehrliche !--Kunden-- ist es ein \--Schock-...,Für ehrliche Kunden ist es ein Schock\nZahl de...,[Für ehrliche !--Kunden-- ist es ein \--Schock...,"[Für ehrliche Kunden ist es ein Schock, Zahl d...",{'dependency_distance_mean': 2.704419452574911...



An dieser Stelle sind die Daten soweit vorbereitet um im [DataExploration](./DataExploration.ipynb) weiter untersucht zu werden.

In [11]:
df.to_pickle('data_prepared_frame.pkl')

,ArtikelId,Text,Text_unmarked,Sentences_marked,Sentences_unmarked
0,ArtikelId: 0001_0001,"""Ross Perot wäre vielleicht ein prächtiger \--...","""Ross Perot wäre vielleicht ein prächtiger Dik...","[""Ross Perot wäre vielleicht ein prächtiger \-...","[""Ross Perot wäre vielleicht ein prächtiger Di..."
1,ArtikelId: 0001_0002,IBM und Siemens gelten nicht mehr als Schimpfw...,IBM und Siemens gelten nicht mehr als Schimpfw...,[IBM und Siemens gelten nicht mehr als Schimpf...,[IBM und Siemens gelten nicht mehr als Schimpf...
2,ArtikelId: 0001_0003,Wechselspiel von Dramatisierung und Ignoranz\n...,Wechselspiel von Dramatisierung und Ignoranz\n...,"[Wechselspiel von Dramatisierung und Ignoranz,...","[Wechselspiel von Dramatisierung und Ignoranz,..."
3,ArtikelId: 0001_0004,Im \--Blickpunkt--:\nErmittlungen gegen \--Aut...,Im Blickpunkt:\nErmittlungen gegen Autonome\nS...,"[Im \--Blickpunkt--:, Ermittlungen gegen \--Au...","[Im Blickpunkt:, Ermittlungen gegen Autonome, ..."
4,ArtikelId: 0001_0005,Für ehrliche !--Kunden-- ist es ein \--Schock-...,Für ehrliche Kunden ist es ein Schock\nZahl de...,[Für ehrliche !--Kunden-- ist es ein \--Schock...,"[Für ehrliche Kunden ist es ein Schock, Zahl d..."
